## Noise Estimation Brute Force Approach

For a single noise value, have multiple simulations for that and that will show the distribution of possible values at that noise level
-  Note that each noise value samples from a distribution already, and the same noise value is going to have different results each time due to this
- Use MonteCarlo from monte_carlo.py
- In 2024PHMTutorial, see monte_carlo example

In [ ]:
from progpy.models import ThrownObject
import numpy as np
import matplotlib.pyplot as plt
from progpy.predictors import MonteCarlo
from scipy.optimize import minimize

# Pick a process and measurement noise as a goal --> then use that as "real" data
PROCESS_NOISE = 1.8
MEASUREMENT_NOISE = 0.4

m = ThrownObject(process_noise=PROCESS_NOISE, measurement_noise=MEASUREMENT_NOISE)
initial_state = m.initialize()

simulated_results = m.simulate_to(8, save_freq=1)

times = simulated_results.times
inputs = simulated_results.inputs
outputs = simulated_results.outputs

def future_loading(t, x=None):
    return {}

STEP_SIZE = 1
NUM_SAMPLES = 100
PREDICTION_HORIZON = times[-1]
SAVE_FREQ = 1

og_mc = MonteCarlo(m)
og_mc_results = og_mc.predict(initial_state, future_loading, dt=STEP_SIZE, horizon=PREDICTION_HORIZON, save_freq=SAVE_FREQ, n_samples=NUM_SAMPLES)

In [ ]:
# Like estimate_params
PROCESS_NOISE_GUESS = 3
MEASUREMENT_NOISE_GUESS = 2

m = ThrownObject(process_noise=PROCESS_NOISE_GUESS, measurement_noise=MEASUREMENT_NOISE_GUESS)
initial_state = m.initialize()

mc = MonteCarlo(m)
mc_results = mc.predict(initial_state, future_loading, dt=STEP_SIZE, horizon=PREDICTION_HORIZON, save_freq=SAVE_FREQ, n_samples=NUM_SAMPLES)
mc_results.states

In [ ]:
# TODO: Put a solo color lighter alpha. Also plot the average real data and that distribution.

for i in range(NUM_SAMPLES):
    data_pos = [state['x'] for state in mc_results.states[i]]
    plt.plot(mc_results.times[:len(data_pos)], data_pos)

# Plot model with no noise
m = ThrownObject(process_noise=0, measurement_noise=0)
simulated_results = m.simulate_to(times[-1], save_freq=1)
no_noise_pos = [state['x'] for state in simulated_results.outputs]
plt.plot(times, no_noise_pos, color="black", linestyle="--", label="No noise simulation")

# Plot real data --> TODO: show mean
pos = [state['x'] for state in outputs]
plt.plot(times, pos, color="blue", linestyle="--", label="Real data")

# Plot MC simulation mean
mc_mean_pos = [state['x'] for state in mc_results.states.mean]
plt.plot(times, mc_mean_pos, color="red", linestyle="--", label="Mean MC simulation")

error = np.mean((np.array(pos) - np.array(mc_mean_pos))**2)
print("MSE error: ", error)

plt.legend()
plt.title("Monte Carlo simulation for one noise value of Thrown Object")
plt.xlabel("Time (s)")
plt.ylabel("Position (m)")
plt.show()

## Distrbutions from Brute Force Approach
- At one time step (like the last one of 8 or let's say 6), get the distribution and compare to the data (which is also a distribution and compare that - get the error value between them and use that to optimize)
- With other noise values (like 5 or 10% below or above optimized number), see how sensitivity changes (mean and variance) after the optimization
   - Helps user understand how precise they need to be with noise number

In [ ]:
# Guess noise distrbution at time step 6
dist = []
for i in mc_results.states:
    dist.append(i[5]['x'])

# OG Distrbution at time step 6
og_dist = []
for i in og_mc_results.states:
    og_dist.append(i[5]['x'])

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].hist(og_dist)
axes[1].hist(dist, color="tab:orange")

axes[0].set_title("OG distribution")
axes[0].set_xlabel("Position (m)")
axes[0].set_ylabel("Count")

axes[1].set_title("Guess distribution")
axes[1].set_xlabel("Position (m)")
axes[1].set_ylabel("Count")

fig.suptitle("Monte Carlo simulations at time step 6")
plt.tight_layout()
plt.show()

In [ ]:
# Calculate errors

# TODO: calculate mean and std and minimize the difference between real data and monte carlo
# Could possibly specify the frequency at which we compare (like a eval freq) instead of doing at every time step (could also investigate
# coarse to fine optimization section)

# TODO 2: look at other measures of distributional similarity - see link in chat

import numpy as np

og_arr = np.array(og_dist)
arr = np.array(dist)

# 1. Mean Squared Error (MSE)
mse = np.mean((og_arr - arr) ** 2)

# 2. Mean Absolute Error (MAE)
mae = np.mean(np.abs(og_arr - arr))

# 3. Euclidean Distance
euclidean_distance = np.sqrt(np.sum((og_arr - arr) ** 2))

# Print the results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Euclidean Distance: {euclidean_distance}")


In [ ]:
# def optimization_fcn(params):
#     error = np.mean((og_arr - arr) ** 2)
#     print(f"Error: {error}, PROCESS_NOISE_GUESS: {PROCESS_NOISE_GUESS}, MEASUREMENT_NOISE_GUESS: {MEASUREMENT_NOISE_GUESS}")

#     return error

# initial_guess = [PROCESS_NOISE_GUESS, MEASUREMENT_NOISE_GUESS]
# res = minimize(optimization_fcn, initial_guess, bounds=((0, 1), (0, 1)))

# if res.success:
#     print("Optimization converged successfully.")
# else:
#     print("Optimization did not converge.")

# print("Optimized process noise: ", res.x[0])
# print("Optimized measurement noise: ", res.x[1])


## Coarse to fine optimization
- Running a few samples (either running 15 mc simulations or running to time 2 seconds)
- Measure runtime, computational power, accuracy of results (since we know our actual noise values since we made the data)